# **Agents and Tools with LangGraph**

### Nihar Lohar 
### BTech Artificial Intelligence 



In [ ]:
!pip install langgraph==0.3.2 mistralai pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.5/440.5 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 24.0 MB/s eta 0:00:00


In [ ]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")
api_key = os.environ["MISTRAL_API_KEY"]

Enter your Mistral API key: ··········


In [ ]:
import json
import pandas as pd
from mistralai import Mistral
import functools
from langgraph.graph import StateGraph, END

In [ ]:
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

df = pd.DataFrame(data)

In [ ]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
      return json.dumps(
          {
              'status': df[df.transaction_id == transaction_id].payment_status.item()
          }
      )
    return json.dumps({'error': 'Transaction not found'})

def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
      return json.dumps(
          {
              'date': df[df.transaction_id == transaction_id].payment_date.item()
          }
      )
    return json.dumps({'error': 'Transaction not found'})
def retrieve_payment_amount(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
      return json.dumps(
          {
              'amount': df[df.transaction_id == transaction_id].payment_amount.item()
          }
      )
      return json.dumps({'error': 'Transaction not found'})

names_to_functions = {
    "retrieve_payment_status": retrieve_payment_status,
    "retrieve_payment_date": retrieve_payment_date,
    "retrieve_payment_amount": retrieve_payment_amount
}

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction ID."
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction ID."
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_amount",
            "description": "Get payment amount of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction ID."
                    }
                },
                "required": ["transaction_id"],
            },
        },
    }
]


In [ ]:
model = "mistral-small-2506"
client = Mistral(api_key=api_key)

In [ ]:
State = dict

In [ ]:
def planner(state: State) -> State:
    messages = state.get("messages", [])
    response = client.chat.complete(
        messages=messages,
        model=model,
        tools=tools,
        tool_choice="any",
    )
    state["messages"].append(response.choices[0].message)
    state["tool_calls"] = response.choices[0].message.tool_calls
    return state


def execute_tools(state: State) -> State:
    tool_calls = state.get("tool_calls", [])
    messages = state.get("messages", [])
    tool_results = []

    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_params = json.loads(tool_call.function.arguments)

        if function_name in names_to_functions:
            # Pass the df DataFrame to the function
            function_result = names_to_functions[function_name](df=df, **function_params)
            tool_results.append({
                "role": "tool",
                "name": function_name,
                "content": function_result,
                "tool_call_id": tool_call.id
            })
        else:
            tool_results.append({
                "role": "tool",
                "name": function_name,
                "content": json.dumps({"error":  f"Tool{function_name}not found"}),
                "tool_call_id": tool_call.id
            })

    messages.extend(tool_results)
    state["messages"] = messages
    state["tool_calls"] = None
    return state


def final_response(state: State) -> State:
    response = client.chat.complete(
        messages=state["messages"],
        model=model,
    )
    state["messages"].append(response.choices[0].message)
    return state

def check_tool_needed(state: State) -> str:
    return "call_tool" if state.get("tool_calls") else "final_answer"

In [ ]:
graph_builder = StateGraph(State)
graph_builder.set_entry_point("planner")

graph_builder.add_node("planner", planner)
graph_builder.add_node("call_tool", execute_tools)
graph_builder.add_node("final_answer", final_response)

graph_builder.add_conditional_edges("planner", check_tool_needed,{
   "call_tool":"call_tool",
   "final_answer" : "final_answer"
})

graph_builder.add_conditional_edges("call_tool", check_tool_needed,{
   "call_tool":"call_tool",
   "final_answer" : "final_answer"
})

graph_builder.add_edge("final_answer",END)

graph = graph_builder.compile()

In [ ]:
def run_query(user_query: str, with_system: bool = True):
    messages = []
    if with_system:
        messages.append(
            {
                "role": "system",
                "content": "You are a helpful assistant that provides imofrmation about payment transactions."
            }
        )
    messages.append(
        {
            "role": "user",
            "content": user_query,
        }
    )
    state = {"messages":messages}
    result = graph.invoke(state)




    return result["messages"][-1].content

In [ ]:
user_query = "What's the status and amount and date of my transaction T1001?"
# user_query = "When was transaction T1005 made?"
# user_query = "who is prime minister of india"

print(run_query(user_query))


The status of your transaction T1001 is Paid. The amount is $125.5 and the date of the transaction is October 5, 2021.
